In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import wrangle

In [4]:
df = acquire.get_zillow_data()

Through domain research and choosing features that have enough data (low null values, less than 20%), I've determined these columns to be the most practical to utilize in the data science pipeline for the purpose of predicting tax assessed home values through regression modeling. 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    52441 non-null  int64  
 1   parcelid                      52441 non-null  int64  
 2   bedroomcnt                    52441 non-null  float64
 3   bathroomcnt                   52441 non-null  float64
 4   calculatedfinishedsquarefeet  52359 non-null  float64
 5   taxvaluedollarcnt             52440 non-null  float64
 6   yearbuilt                     52325 non-null  float64
 7   taxamount                     52437 non-null  float64
 8   fips                          52441 non-null  float64
 9   assessmentyear                52441 non-null  float64
 10  landtaxvaluedollarcnt         52440 non-null  float64
 11  lotsizesquarefeet             52072 non-null  float64
 12  latitude                      52441 non-null  float64
 13  l

In [6]:
df.isna().sum()

Unnamed: 0                        0
parcelid                          0
bedroomcnt                        0
bathroomcnt                       0
calculatedfinishedsquarefeet     82
taxvaluedollarcnt                 1
yearbuilt                       116
taxamount                         4
fips                              0
assessmentyear                    0
landtaxvaluedollarcnt             1
lotsizesquarefeet               369
latitude                          0
longitude                         0
dtype: int64

To not lose too much data, I'm looking at columns with many null values and how many null values they share with other columns

In [7]:
df['calculatedfinishedsquarefeet'].isna().sum() & df['lotsizesquarefeet'].isna().sum() & df['yearbuilt'].isna().sum()

80

square feet, lot size, and year built share a lot of null rows. This is good news when considering dropping columns with null values.

In [8]:
df['lotsizesquarefeet'].isna().sum() & df['taxamount'].isna().sum(), df['calculatedfinishedsquarefeet'].isna().sum() & df['taxamount'].isna().sum()

(0, 0)

most of tax amount nulls are also square feet nulls so again, feel better about dropping these records

In [9]:
(df.isnull().sum() / len(df)).round(3) 
#this shows the percentage of null values per column rounded to the 3rd decimal place

Unnamed: 0                      0.000
parcelid                        0.000
bedroomcnt                      0.000
bathroomcnt                     0.000
calculatedfinishedsquarefeet    0.002
taxvaluedollarcnt               0.000
yearbuilt                       0.002
taxamount                       0.000
fips                            0.000
assessmentyear                  0.000
landtaxvaluedollarcnt           0.000
lotsizesquarefeet               0.007
latitude                        0.000
longitude                       0.000
dtype: float64

In [11]:
df.assessmentyear.value_counts()

2016.0    52441
Name: assessmentyear, dtype: int64

we can see that all columns have less than 1% null (missing) values. This is favorable to dropping nulls

In [8]:
df.isna().mean().sum()
#shows the percentage of missing values in the entire dataframe. 

0.01705589254866659

If all nulls are dropped, we will lose 1.7% of the data. With this low percentage and seeing how most records with nulls have nulls across other columns, I determine dropping records with nulls is the most appropriate solution

In [12]:
df = wrangle.wrangle_zillow(df)

In [10]:
df

,Unnamed: 0,parcelid,bedrooms,bathrooms,home_size,value,year,tax,fips,land_value,lot_size,latitude,longitude,county
4,4,11324547,4.0,2.0,3633,296425,2005,6941.39,6037,74104,9826,34560018.0,-118169806.0,los_angeles
7,7,11585547,3.0,2.0,2077,646760,1926,7924.68,6037,436568,6490,34012977.0,-118479243.0,los_angeles
18,18,12716947,3.0,1.0,1244,169471,1950,2532.88,6037,61431,6021,33953559.0,-118083855.0,los_angeles
19,19,12757147,3.0,2.0,1300,233266,1950,3110.99,6037,155851,4917,33897134.0,-118102953.0,los_angeles
20,20,12757947,3.0,2.0,1222,290492,1951,3870.25,6037,172653,5500,33889894.0,-118100732.0,los_angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2152853,2152853,13917885,4.0,2.0,1987,259913,1955,3175.66,6059,169147,6100,33851932.0,-117934693.0,orange
2152855,2152855,14762130,3.0,2.5,1809,405547,2012,4181.10,6059,148031,4884,33634876.0,-117833446.0,orange
2152858,2152858,14356230,4.0,3.0,2262,960756,2015,13494.52,6059,564569,3220,33687201.0,-117759785.0,orange
2152859,2152859,14157725,4.0,4.5,3127,536061,2014,6244.16,6059,147419,8930,33929748.0,-117801513.0,orange


In [11]:
df = wrangle.wrangle_locs(df)

KeyboardInterrupt: 

In [13]:
df.drop(columns= 'Unnamed: 0', inplace = True)

In [14]:
df

,parcelid,bedrooms,bathrooms,home_size,value,year,tax,fips,land_value,lot_size,latitude,longitude,county
0,14297519,4.0,3.5,3100,1023282,1998,11013.72,6059,537569,4506,33634931.0,-117869207.0,orange
1,17052889,2.0,1.0,1465,464000,1967,5672.48,6111,376000,12647,34449266.0,-119281531.0,ventura
2,14186244,3.0,2.0,1243,564778,1962,6488.30,6059,479489,8432,33886168.0,-117823170.0,orange
3,12177905,4.0,3.0,2376,145143,1970,1777.51,6037,36225,13038,34245180.0,-118240722.0,los_angeles
4,12095076,4.0,3.0,2962,773303,1950,9516.26,6037,496619,63000,34145202.0,-118179824.0,los_angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,4.0,2.0,1633,346534,1962,4175.08,6037,221068,4630,33870815.0,-118070858.0,los_angeles
52437,11000655,2.0,2.0,1286,354621,1940,4478.43,6037,283704,47405,34245368.0,-118282383.0,los_angeles
52438,17239384,4.0,2.0,1612,67205,1964,1107.48,6111,16522,12105,34300140.0,-118706327.0,ventura
52439,12773139,3.0,1.0,1032,49546,1954,876.43,6037,16749,5074,34040895.0,-118038169.0,los_angeles


In [15]:
df.value.mean()

508813.64022646466

seeing that tax assessed value mean is very different based on the assessment year, I am dropping 2014 and 2015 assessment year records from the dataframe for consistency

In [ ]:
#df.assessmentyear.value_counts()

Now that all records have the same assessment year (2016), I can drop this column

In [ ]:
#df.drop(columns = ['assessmentyear'], inplace=True)

In [16]:
df

,parcelid,bedrooms,bathrooms,home_size,value,year,tax,fips,land_value,lot_size,latitude,longitude,county
0,14297519,4.0,3.5,3100,1023282,1998,11013.72,6059,537569,4506,33634931.0,-117869207.0,orange
1,17052889,2.0,1.0,1465,464000,1967,5672.48,6111,376000,12647,34449266.0,-119281531.0,ventura
2,14186244,3.0,2.0,1243,564778,1962,6488.30,6059,479489,8432,33886168.0,-117823170.0,orange
3,12177905,4.0,3.0,2376,145143,1970,1777.51,6037,36225,13038,34245180.0,-118240722.0,los_angeles
4,12095076,4.0,3.0,2962,773303,1950,9516.26,6037,496619,63000,34145202.0,-118179824.0,los_angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,4.0,2.0,1633,346534,1962,4175.08,6037,221068,4630,33870815.0,-118070858.0,los_angeles
52437,11000655,2.0,2.0,1286,354621,1940,4478.43,6037,283704,47405,34245368.0,-118282383.0,los_angeles
52438,17239384,4.0,2.0,1612,67205,1964,1107.48,6111,16522,12105,34300140.0,-118706327.0,ventura
52439,12773139,3.0,1.0,1032,49546,1954,876.43,6037,16749,5074,34040895.0,-118038169.0,los_angeles


In [ ]:
# df.rename(columns={'landtaxvaluedollarcnt': 'land_value', 'lotsizesquarefeet': 'lot_size', 'square_feet': 'home_size'}, inplace=True)

In [ ]:
# df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# df

In [ ]:
# df['land_value'] = df.land_value.astype(int)

In [ ]:
# df['lot_size'] = df.lot_size.astype(int)

### Univariate Exploration

In [ ]:
df.bedrooms.value_counts().sort_values()

In [ ]:
df.bathrooms.value_counts().sort_values()

In [ ]:
# put this in wrangle function 

df = df[df.lot_size > 500]
df = df[df.home_size > 500]
df = df[df.value > 10000]

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
print()
printmd("**Zillow Data (Min, Max, Average)**")
print("--------------------------------")
printmd("**Tax Assessed Value of Home**")
printmd('*Maximum Tax Assessed Value: {:,}*'
     .format(df['value'].max()))
printmd('*Minimum Tax Assessed Value: {:,}*'
     .format(df['value'].min()))
printmd('*Average Tax Assessed Value: {:,}*'
     .format(round(df['value'].mean())))
print("--------------------------------")
printmd("**Tax Amount of Home**")
printmd('*Maximum Tax Amount of Home: {:,}*'
     .format(round(df['tax'].max())))
printmd('*Minimum Tax Amount of Home: {:,}*'
     .format(df['tax'].min()))
printmd('*Average Tax Amount of Home: {:,}*'
     .format(round(df['tax'].mean())))
print("--------------------------------")
printmd("**Home Size in Square Feet**")
printmd('*Maximum Home Size: {:,} square feet*'
     .format(df['home_size'].max()))
printmd('*Minimum Home Size: {:,} square feet*'
     .format(df['home_size'].min()))
printmd('*Average Home Size: {:,} square feet*'
     .format(round(df['home_size'].mean())))
print("--------------------------------")
printmd("**Lot Size in Square Feet**")
printmd('*Maximum Lot Size: {:,} square feet*'
     .format(df['lot_size'].max()))
printmd('*Minimum Lot Size: {:,} square feet*'
     .format(df['lot_size'].min()))
printmd('*Average Lot Size: {:,} square feet*'
     .format(round(df['lot_size'].mean())))
print("--------------------------------")
printmd("**Tax Assessed Land Value**")
printmd('*Maximum Land Value: {:,}*'
     .format(df['land_value'].max()))
printmd('*Minimum Land Value: {:,}*'
     .format(df['land_value'].min()))
printmd('*Average Land Value: {:,}*'
     .format(round(df['land_value'].mean())))

In [ ]:
plt.figure(figsize=(10, 4))
sns.set_palette('Set2', 8)
plt.title('bedroom count')
sns.countplot(x = 'bedrooms', data= df)

In [ ]:
plt.figure(figsize=(14, 5))
sns.set_palette('Set2', 20)
sns.countplot(x='bathrooms', data=df)
plt.title('bathroom count')

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

from math import sqrt
from scipy import stats

In [ ]:
# make this a function
train_and_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_and_validate, test_size=.3, random_state=123)

In [ ]:
train.info()

In [ ]:
orange_train = train.copy()

In [ ]:
ventura_train = train.copy()

In [ ]:
la_train = train.copy()

In [ ]:
orange_train = orange_train[orange_train.county == 'orange']

In [ ]:
orange_train

In [ ]:
ventura_train = ventura_train[ventura_train.county == 'ventura']

In [ ]:
la_train = la_train[la_train.county == 'los_angeles']

In [ ]:
ventura_train.to_csv("ventura_locations.csv")
la_train.to_csv("la_locations.csv")
orange_train.to_csv("orange_locations.csv")

In [ ]:
ventura_train.info()

In [ ]:
ventura_train['latitude'] = ventura_train.latitude.astype(str)

In [ ]:
ventura_train['latitude'].dtype

In [ ]:
ventura_train['latitude'] = np.where(ventura_train['latitude']==' ',0, ventura_train['latitude']).astype(float)

In [ ]:
ventura_train['longitude'] = ventura_train.longitude.astype(str)

In [ ]:
ventura_train['longitude'] = ventura_train['longitude'].str.rstrip('.0') 

In [ ]:
ventura_train['longitude']

In [ ]:
ventura_train['longitude'] = np.where(ventura_train['longitude']==' ',0, ventura_train['longitude']).astype(float)

In [ ]:
import geopy

In [ ]:
def get_zipcode(ventura_train, geolocator, lat_field, lon_field):
    location = geolocator.reverse((ventura_train['latitude'], ventura_train['longitude']))
    return location.raw['address']['postcode']

geolocator = geopy.Nominatim(user_agent='mlshiben')

In [ ]:
ventura_train.get('latitude')

In [ ]:
import geocoder

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=50)

In [ ]:
def geo_rev(ventura_train):
    g = geocoder.osm([ventura_train.latitude, ventura_train.longitude], method='reverse').json
    if g:
        return g.get('postal')
    else:
        return 'no postal'

ventura_zips = ventura_train[['latitude', 'longitude']].apply(geo_rev, axis=1)


In [ ]:
g = geocoder.osm(ventura_train['latitude'], ventura_train['longitude']), method='reverse')
g.json['postal']

In [ ]:
zipcodes = ventura_train.apply(get_zipcode, geolocator=geolocator, lat_field ='latitude', lon_field = 'longitude')